### Tratamento dos dados para Regressão dos microdados da PNAD COVID

Tratamento das variáveis que serão inseridas na regressão em painel

Pacotes para manipulação e regressão

In [1]:
import pandas as pd
import numpy as np
import datetime

pnad_covid = pd.read_parquet('../Dados/pnad_covid/pnad_covid_filtrada.parquet')

### Tratamento das variáveis do modelo

segundo nota técnica do IPEA

**Variáveis da PNAD Covid que serão filtradas:**

<img src="../Anexos/var_estudo.png" width="600" height="500">

Faixa etária (A002)

In [2]:
#grupos de 10 a partir do 20 e até o 80

#Grupos de 10 a partir do 20 e até o 80
pnad_covid['grupo_etario'] = pnad_covid['A002'].apply(lambda x:
                                                      '14-20' if x < 19
                                                      else '20-29' if x < 29
                                                      else '30-39' if x < 39
                                                      else '40-49' if x < 49
                                                      else '50-59' if x < 59
                                                      else '60-69' if x < 69
                                                      else '70-79' if x < 79
                                                      else '80 ou mais' if x >= 80
                                                      else x)


pnad_covid['grupo_etario'] = pd.Categorical(pnad_covid['grupo_etario'], categories=[
    '14-20',
    '20-29'
    '30-39'
    '40-49'
    '50-59'
    '60-69'
    '70-79'
    '80 ou mais'    
], ordered=True)

Sexo (A003)

In [3]:
#mulher recebe valor 0 e homem valor 1

#pnad_covid['A003'] = pnad_covid['A003'].map({'Mulher':0, 'Homem':1})

#transformando a variável em categórica

pnad_covid['A003'] = pd.Categorical(pnad_covid['A003'], categories=[
    'Mulher',
    'Homem'
], ordered=False)


Cor-raça (A004)

In [4]:
#brancos e amarelos como 1 e demais 0

#pnad_covid['A004'] = pnad_covid['A004'].apply(lambda x: 1 if x in ['Branca', 'Amarela'] else 0)

#reagrupar as categorias de cor ou raça para "Brancos e Amarelos", "Pretos, Pardos e Indígenas" e "Ignorado"

pnad_covid['Raça'] = pnad_covid['A004'].apply(lambda x:
                                                'Brancos e Amarelos' if x == 'Branca' or x == 'Amarela' 
                                                else 'Pretos, Pardos e Indígenas' if x == 'Preta' or x == 'Parda' or x == 'Indígena'
                                                else 'Ignorado' if x == 'Ignorado'
                                                else x)

#tranformar em categórica

pnad_covid['Raça'] = pd.Categorical(pnad_covid['Raça'], categories=[
    'Brancos e Amarelos',
    'Pretos, Pardos e Indígenas',
    'Ignorado'
], ordered=False)

Escolaridade (A005)

In [5]:
#Classificação da escolaridade conforme variável

# pnad_covid['A005'] = pd.Categorical(pnad_covid['A005'], categories=[
#     'Sem instrução',
#     'Fundamental incompleto',
#     'Fundamental completa',
#     'Médio incompleto',
#     'Médio completo',
#     'Superior incompleto',
#     'Superior completo',
#     'Pós-graduação, mestrado ou doutorado'
# ], ordered=True)

#Reagrupar escolaridade em "Sem instrução", "Fundamental Completo e Médio Incompleto", "Médio Completo e Superior Incompleto", "Superior Completo"

pnad_covid['Escolaridade'] = pnad_covid['A005'].apply(lambda x:
                                                        'Sem instrução' if x == 'Sem instrução'
                                                        else 'Sem instrução' if x == 'Fundamental incompleto'
                                                        else 'Fundamental Completo e Médio Incompleto' if x == 'Fundamental completa'
                                                        else 'Fundamental Completo e Médio Incompleto' if x == 'Médio incompleto'
                                                        else 'Médio Completo e Superior Incompleto' if x == 'Médio completo'
                                                        else 'Médio Completo e Superior Incompleto' if x == 'Superior incompleto'
                                                        else 'Superior Completo' if x == 'Superior completo'
                                                        else 'Superior Completo' if x == 'Pós-graduação, mestrado ou doutorado'
                                                        else x)

#tranformar em categórica

pnad_covid['Escolaridade'] = pd.Categorical(pnad_covid['Escolaridade'], categories=[
    'Sem instrução',
    'Fundamental Completo e Médio Incompleto',
    'Médio Completo e Superior Incompleto',
    'Superior Completo'
], ordered=True)

Formalidade (Composta por C007-Setor e C007B- formalidade)

In [6]:
#setor privado sem carteira, setor publico não estatutario e sem carteira, sem contribuição para a previdencia = Informal

def formalidade(row):
    if row['C007'] == "Empregado do setor privado":
        if row['C007B'] == 'Sim, tem carteira de trabalho assinada':
            return 'Formal'
        elif row['C007B'] == 'Sim, é servidor público estatutário':
            return 'Formal'
        elif row['C007B'] == 'Não':
            return 'Informal'
        else:
            return 'Não identificado'
            
    elif row['C007'] == "nan":
        if row['C007B'] == 'Sim, tem carteira de trabalho assinada':
            return 'Formal'
        elif row['C007B'] == 'Sim, é servidor público estatutário':
            return 'Formal'
        elif row['C007B'] == 'Não':
            return 'Informal'
        else:
            return 'Não identificado'

    elif row['C007'] == "Conta própria":
        return 'Conta própria'

    elif row['C007'] == "Empregado do setor público (inclusive empresas de economia mista)":
        if row['C007B'] == 'Sim, tem carteira de trabalho assinada':
            return 'Formal'
        elif row['C007B'] == 'Sim, é servidor público estatutário':
            return 'Formal'
        elif row['C007B'] == 'Não':
            return 'Informal'
        else:
            return 'Não identificado'

    elif row['C007'] == "Policial militar ou bombeiro militar":
        return 'Militar'

    elif row['C007'] == "Trabalhador doméstico (empregado doméstico, cuidados, babá)":
        if row['C007B'] == 'Sim, tem carteira de trabalho assinada':
            return 'Formal'
        elif row['C007B'] == 'Sim, é servidor público estatutário':
            return 'Formal'
        elif row['C007B'] == 'Não':
            return 'Informal'
        else:
            return 'Não identificado'
        
    elif row['C007'] == "Empregador":
        return 'Empregador'
    elif row['C007'] == "Militar do exercito, marinha ou aeronáutica":
        return 'Militar'
    
    elif row['C007'] == "Trabalhador familiar não remunerado em ajuda a membro do domicílio ou parente":
        return 'Trabalhador familiar'

    elif row['C007'] == "Estava fora do mercado de trabalho (fazia apenas afazeres domésticos, cuidados de pessoas ou produção para próprio consumo)":
        return 'Fora do mercado de trabalho'

In [7]:
#criar coluna de formalidade com base em duas colunas da pnad_covid

pnad_covid['formalidade'] = pnad_covid.apply(formalidade, axis=1)

In [12]:
#reagrupar coluna de formalidade como formal, informal

pnad_covid['formalidade'] = pnad_covid['formalidade'].apply(lambda x:
                                                            'Formal' if x == 'Formal'
                                                            else 'Informal' if x == 'Informal'
                                                            else 'Informal' if x == 'Conta própria'
                                                            else 'Informal' if x == 'Militar'
                                                            else 'Informal' if x == 'Empregador'
                                                            else 'Informal' if x == 'Trabalhador familiar'
                                                            else 'Fora do mercado de trabalho' if x == 'Fora do mercado de trabalho'
                                                            else x)

#tranformar em categórica

pnad_covid['formalidade'] = pd.Categorical(pnad_covid['formalidade'], categories=[
    'Formal',
    'Informal',
    'Fora do mercado de trabalho'
], ordered=False)

Atividade(setor) (C007D)

In [14]:
#Identificadas como agricultura, comércio, setor público, serviços e industria

def setor(row):
    if row['C007D'] == 'Agricultura, pecuária, produção florestal e pesca':
        return 'Agricultura'
    elif row['C007D'] == 'Comércio no atacado e varejo;':
        return 'Comércio'
    elif row['C007D'] == 'Administração pública (governo federal, estadual e municipal)':
        return 'Setor público'
    elif row['C007D'] in ['Educação', 
                          'Construção',
                          'Saúde humana e assistência social',
                          'Serviço doméstico remunerado (será imputado da posição na ocupação)',
                          'Serviço de alimentação (bares, restaurantes, ambulantes de alimentação)',
                          'Atividades de locação de mão de obra, segurança, limpeza, paisagismo e teleatendimento',
                          'Escritórios de advocacia, engenharia, publicidade e veterinária (Atividades profissionais, científicas e técnicas)',
                          'Reparação de veículos automotores e motocicletas',
                          'Transporte de mercadorias',
                          'Cabeleireiros, tratamento de beleza e serviços pessoais',
                          'Transporte de passageiros',
                          'Informação e comunicação (jornais, rádio e televisão, telecomunicações e informática)',
                          'Bancos, atividades financeiras e de seguros',
                          'Fornecimento de eletricidade e gás, água, esgoto e coleta de lixo',
                          'Atividade artísticas, esportivas e de recreação',
                          'Atividades imobiliárias',
                          'Organizações religiosas, sindicatos e associações',
                          'Hospedagem (hotéis, pousadas etc.)',
                          'Armazenamento, correios e serviços de entregas']:
        return 'Serviços'
    elif row['C007D'] in ['Indústria da transformação (inclusive confecção e fabricação caseira)',
                            'Extração de petróleo, carvão mineral, minerais metálicos, pedra, areia, sal etc.']:
        return 'Indústria'
    elif row['C007D'] == 'Outro':
        return 'Outros'

In [15]:
pnad_covid['setor'] = pnad_covid.apply(setor, axis=1)

In [18]:
pnad_covid['setor'] = pd.Categorical(pnad_covid['setor'], categories=[
    'Serviços',
    'Agricultura',
    'Comércio',
    'Indústria',
    'Outros',
    'Setor público'
], ordered=False)

Ocupação (C001, C002, C003)

In [20]:
def ocupacao(row):
    
    if row['C001'] == 'Sim':
        if row['C013'] == 'Sim':
            return 'Pessoa ocupada exercendo atividade de maneira remota'
        
        elif row['C013'] == 'Não':
            return 'Pessoa ocupada não afastada'

    elif row['C001'] == 'Não':
        if row['C002'] == 'Sim':
            if row['C003'] == 'Estava em quarentena, isolamento, distanciamento social ou férias coletivas':
    
                return 'Pessoas afastada devido distanciamento social'

            elif row['C003'] in ['Licença remunerada por motivo de saúde ou acidente da própria pessoa',
                                    'Férias, folga ou jornada de trabalho variável',
                                    'Afastamento do próprio negócio/empresa por motivo de gestação, saúde, acidente, etc., sem ser remunerado por instituto de previdência',
                                    'Outro motivo', 
                                    'Licença maternidade ou paternidade',
                                    'Fatores ocasionais (mau tempo, paralisação nos serviços de transportes, etc.)',
                                    'Outro tipo de licença remunerada (estudo, paternidade, casamento, licença prêmio, etc.)']:
    
                return 'Pessoas afastada por outras razões'
            
            else:
                return row['C003']
        
    else:
        return row['C001']


In [21]:
pnad_covid['Ocupação'] = pnad_covid.apply(ocupacao, axis=1)

In [23]:
#tranformar em categórica

pnad_covid['Ocupação'] = pd.Categorical(pnad_covid['Ocupação'], categories=[
    'Pessoa ocupada exercendo atividade de maneira remota',
    'Pessoa ocupada não afastada',
    'Pessoas afastada devido distanciamento social',
    'Pessoas afastada por outras razões'
], ordered=False)

### Tratamento da variável de tempo

In [25]:
# Definindo a data de referência como o primeiro dia de maio
data_referencia = datetime.datetime(2023, 5, 1)

# Definindo a data final como o último dia de novembro
data_final = datetime.datetime(2023, 11, 30)

# Criando uma lista vazia para armazenar as semanas
semanas = []

# Loop para adicionar as semanas à lista
while data_referencia <= data_final:
    numero_semana = data_referencia.strftime("%V")
    semanas.append(numero_semana)
    data_referencia += datetime.timedelta(weeks=1)

# Imprimindo a lista de semanas

#semanas como int

semanas = [int(i) for i in semanas]

print(semanas)

[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]


In [26]:
df_data = pnad_covid[['Ano','V1013','V1012']].copy()

In [28]:
df_data['Ano'] = df_data['Ano'].astype(int)
df_data['V1013'] = df_data['V1013'].astype(int)
df_data['V1012'] = df_data['V1012'].astype(int)

data_ordenada = df_data.sort_values(by=['Ano','V1013','V1012'], ascending=[True, True, True])

data_semana = data_ordenada.drop_duplicates(subset=['Ano','V1013','V1012'], keep='first')

In [34]:
data_semana['semana'] = semanas[0:28]

/tmp/ipykernel_1140/1178876510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_semana['semana'] = semanas[0:28]


In [35]:
#join columns Ano V1013 V1012
data_semana['data_original'] = data_semana['Ano'].astype(str) + '-' + data_semana['V1013'].astype(str) + '-' + data_semana['V1012'].astype(str)

/tmp/ipykernel_1140/4059033413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_semana['data_original'] = data_semana['Ano'].astype(str) + '-' + data_semana['V1013'].astype(str) + '-' + data_semana['V1012'].astype(str)


In [39]:
#dict where key is data_original and value is semana

dict_semana = data_semana.set_index('data_original')['semana'].to_dict()

In [40]:
pnad_covid['data_formatada'] = pnad_covid['Ano'].astype(str) + '-' + pnad_covid['V1013'].astype(str) + '-' + pnad_covid['V1012'].astype(str)

In [41]:
pnad_covid['semana_dt'] = pnad_covid['data_formatada'].map(dict_semana)

### Dummy de Flexibilização

In [43]:
df_datas_flex = pd.read_excel('../Anexos/datas_flexibilização.xlsx')

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [92]:
df_datas_flex['semana'] = pd.to_datetime(df_datas_flex['Data'].values, format='%d/%m/%Y').strftime('%U')


In [93]:
df_datas_flex.head()

,UF,Data,semana
0,AC,26/06/2020,25
1,AL,22/06/2020,25
2,AM,01/06/2020,22
3,AP,13/06/2020,23
4,BA,07/07/2020,27


Criação de variáveis indicadoras: Cada variável indicadora terá o valor 1 se a data corresponder à flexibilização de um determinado estado e 0 caso contrário.

In [94]:
df_datas_flex.set_index('UF', inplace=True)

In [95]:
df_dummy_flex = pd.get_dummies(df_datas_flex['semana'])

In [96]:
df_dummy_flex.head()

,12,15,17,19,20,21,22,23,24,25,27,28
UF,,,,,,,,,,,,
AC,False,False,False,False,False,False,False,False,False,True,False,False
AL,False,False,False,False,False,False,False,False,False,True,False,False
AM,False,False,False,False,False,False,True,False,False,False,False,False
AP,False,False,False,False,False,False,False,True,False,False,False,False
BA,False,False,False,False,False,False,False,False,False,False,True,False


In [97]:
#transformar df_dummy_flex em dict

dict_dummy_flex = df_dummy_flex.to_dict('index')

In [98]:
uf_dict = {
    'Acre': 'AC',
    'Alagoas': 'AL',
    'Amapá': 'AP',
    'Amazonas': 'AM',
    'Bahia': 'BA',
    'Ceará': 'CE',
    'Distrito Federal': 'DF',
    'Espírito Santo': 'ES',
    'Goiás': 'GO',
    'Maranhão': 'MA',
    'Mato Grosso': 'MT',
    'Mato Grosso do Sul': 'MS',
    'Minas Gerais': 'MG',
    'Pará': 'PA',
    'Paraíba': 'PB',
    'Paraná': 'PR',
    'Pernambuco': 'PE',
    'Piauí': 'PI',
    'Rio de Janeiro': 'RJ',
    'Rio Grande do Norte': 'RN',
    'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO',
    'Roraima': 'RR',
    'Santa Catarina': 'SC',
    'São Paulo': 'SP',
    'Sergipe': 'SE',
    'Tocantins': 'TO'
}

In [99]:
pnad_covid['sigla_uf'] = pnad_covid['UF'].map(uf_dict)

In [100]:
#concatenar pnad_covid com df_dummy_flex a partir da UF

pnad_covid = pd.merge(pnad_covid, df_dummy_flex, left_on='sigla_uf', right_index=True, suffixes=('', '_flex'))

Agrupamento de dados: Se você possui várias observações para cada semana, pode ser útil realizar um agrupamento dos dados por semana e estado, calculando estatísticas resumo relevantes. Por exemplo, você pode calcular a contagem de casos de COVID-19 ou a taxa de desemprego para cada semana e estado e incluir essas variáveis agregadas no modelo.


### Exportação dos dados

In [101]:
pnad_covid.to_parquet('../Dados/pnad_covid/pnad_covid_tratada.parquet')

: 

In [43]:
#seleção das variáveis principais
df_selecionado = pnad_covid[['semana_dt', 'domicilio_id','A003', 'A004', 'A005', 'grupo_etario', 'setor', 'C001', 'C013']]

In [44]:
df_selecionado.to_parquet('../Dados/pnad_covid/pnad_covid_selecionado.parquet')